In [135]:
import numpy as np
import random
import warnings
import random
import copy
import pandas as pd
import time
from datetime import datetime
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from sympy import symbols, sympify, lambdify

# Configuración warnings
################################################################################
import warnings
warnings.filterwarnings('ignore')

In [136]:
doc2 = open("C:/Users/antoc/Downloads/PortableGit/practica2024/hub_data.txt")
lineas2 = doc2.readlines()

H_0= 70

# Divide cada línea en columnas usando espacio como separador
datos_separados2 = [linea.strip().split(' ') for linea in lineas2]

datos= []
for fila in datos_separados2:
    datos.append(fila)

z = []
for i in range(len(datos)):
      a = float(datos[i][0])
      z.append(a)
z =np.array(z)

H = []
for i in range(len(datos)):
   if datos[i][1] == '':
      a = float(datos[i][2])
      H.append(a)
   else:
      a = float(datos[i][1])
      H.append(a)
H = np.array(H)

desv = []
for i in range(len(datos)):
    a = float(datos[i][2])
    desv.append(a)


### Clase individuo

In [140]:
class Individuo:
    def __len__(self):
            return len(self.elementos)
    
    def __init__(self, grammar, operaciones, verbose=False):

        self.elementos = []
        self.string = []
        # Valor de las variables del individuo
        #self.valor_variables = np.repeat(None, n_variables)
        # Fitness del individuo
        self.fitness = None
        # Valor de la función objetivo
        self.valor_funcion = None

        #Se genera un valor aleatorio para las variables
        random_funcion1 = random.choice(grammar)
        random_funcion2 = random.choice(grammar)
        random_operador = random.choice(operaciones)

        self.elementos.append(random_funcion1)
        self.elementos.append(random_operador)
        self.elementos.append(random_funcion2)

    @staticmethod
    def polix(x):
        return x**x

    @staticmethod
    def poli1(x):
        return (x + 1)

    @staticmethod
    def poli2(x):
        return x**2

    @staticmethod
    def poli3(x):
        return x**3
    # Crear lista con coeficientes
    coeficientes = [random.random() for i in range(5)]
    # Crear lista de funciones constantes
    funciones_c = [lambda x, i=i: Individuo.coeficientes[i] for i in range(5)]
        
    #x deben estar entre paréntesis para no confundir con otros x en el string
    @staticmethod
    def a_string(funcion):
        if funcion == Individuo.polix:
            return '(x)**(x)'
        elif funcion == Individuo.poli1:
            return '((x)+1)'
        elif funcion == Individuo.poli2:
            return '(x)**2'
        elif funcion == Individuo.poli3:
            return '(x)**3'
        for i in range(len(Individuo.coeficientes)):
            if funcion == Individuo.funciones_c[i]:
                    return str(Individuo.coeficientes[i])

    @staticmethod
    def pasar_a_funcion(expresion):
        a_sympy = sympify(expresion)
        a_python = lambdify(Individuo.x, a_sympy, 'numpy')
        return a_python

    @staticmethod
    def composicion(lista):
        modi_int = Individuo.a_string(lista[-1])
        for i in range(2, len(lista) + 1):
            modi_ext = Individuo.a_string(lista[-i])
            modi_int = modi_ext.replace('(x)', '(' + modi_int + ')')
        return modi_int

    def convertir_lista_a_str(self):
        for i in range(len(self.elementos)):
            if callable(self.elementos[i]):
                if i != 0 and callable(self.elementos[i - 1]):
                    self.string = self.string
                elif i < len(self.elementos) - 1 and callable(self.elementos[i + 1]):
                    compo = []
                    for j in range(i, len(self.elementos)):
                        if callable(self.elementos[j]):
                            compo.append(self.elementos[j])

                    self.string += Individuo.composicion(compo)
                else:
                    self.string += Individuo.a_string(self.elementos[i])
            else:
                self.string += self.elementos[i]
        self.string = ''.join(self.string)

    
    
        
    

In [141]:
grammar = [Individuo.polix, Individuo.poli1, Individuo.poli2, Individuo.poli3] + Individuo.funciones_c

operaciones = ['+', '-', '*', '/']

indiv =Individuo(grammar, operaciones)


In [142]:
indiv.convertir_lista_a_str()
print(indiv.elementos)
print(indiv.string)



[<function Individuo.<listcomp>.<lambda> at 0x000001C4969DD940>, '*', <function Individuo.polix at 0x000001C496990A60>]
0.9786360165819347*(x)**(x)
